<a href="https://colab.research.google.com/github/Gyany29/gyany_29/blob/main/Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mount Google Drive

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import Libraries

In [20]:
import numpy as np
import math as mt
import random
import pandas as pd
import sys

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
c=-1
eeta=0.1
max_iter=1000
import warnings
warnings.filterwarnings('ignore')

#Load Data

In [26]:
data = pd.read_csv('/content/drive/MyDrive/assi5.csv')
data.head()

,Date,Time,Load (kW)
0,01.09.2018,00:00,5551.82208
1,NaN,01:00,4983.17184
2,NaN,02:00,4888.39680
3,NaN,03:00,5072.95872
4,NaN,04:00,5196.25980


In [27]:
X1=[]
X2=[]
X3=[]
Y=[]
data=data['Load (kW)']
for i in range(3,data.shape[0]):
     X1.append(data.iloc[i-3])
     X2.append(data.iloc[i-2])
     X3.append(data.iloc[i-1])
     Y.append(data.iloc[i])

In [28]:
data=pd.DataFrame({'L-3':X1,'L-2':X2,'L-1':X3,'L':Y})
data.head()

,L-3,L-2,L-1,L
0,5551.82208,4983.17184,4888.39680,5072.95872
1,4983.17184,4888.39680,5072.95872,5196.25980
2,4888.39680,5072.95872,5196.25980,5641.29720
3,5072.95872,5196.25980,5641.29720,7153.17732
4,5196.25980,5641.29720,7153.17732,7849.80504


#Normalization

In [30]:
def normalize(column):
    c_max,c_min=np.max(column),np.min(column)
    for i in range(len(column)):
        column.iloc[i]=(column.iloc[i]-c_min)/(c_max-c_min)
    return c_max,c_min
x1max,x1min=normalize(data['L-3'])
x2max,x2min=normalize(data['L-2'])
x3max,x3min=normalize(data['L-1'])
ymax,ymin=normalize(data['L'])
data.head()

,L-3,L-2,L-1,L
0,0.397877,0.293800,0.276454,0.310234
1,0.293800,0.276454,0.310234,0.332801
2,0.276454,0.310234,0.332801,0.414254
3,0.310234,0.332801,0.414254,0.690965
4,0.332801,0.414254,0.690965,0.818465


#Missing Values Treatment

In [8]:
data.isnull().sum()

L(T-3)    0
L(T-2)    0
L(T-1)    0
L(T)      0
dtype: int64

#Data Split

In [31]:
X=data[['L-3','L-2','L-1']]
Y=data['L']
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,Y,test_size=0.1,random_state=101)

#Training Using Stochastic Gradient Descent Optimizer

In [10]:
m=[np.random.uniform(-10,10) for i in range (Xtrain.shape[1])]
c=np.random.uniform(-10,10)
epochs=500
for _ in range (epochs):
    for i in range (Xtrain.shape[0]):
     Y,Ypred=Ytrain.iloc[i],sum([Xtrain.iat[i,z]*m[z] for z in range(Xtrain.shape[1])])
     gradm=[-1*Xtrain.iat[i,j]*(Y-Ypred-c) for j in range(Xtrain.shape[1])]
     gradc=-1*(Y-Ypred-c)
     c=c-eeta*gradc
     for j in range (len(m)):
        m[j]=m[j]-eeta*gradm[j]
print (m,c)
m1=m[0]
m2=m[1]
m3=m[2]

[-0.23128366799983116, -0.007745238430485217, 0.9080113730757409] 0.14143639495293983


#Model Prediction on Training Data

In [11]:
y_pred_train = []
for sample in range(Xtrain.shape[0]):
  y_p_tr = (m[0] * Xtrain.iat[sample,0]) + c + (m[1] *Xtrain.iat[sample,1] )+(m[2]*Xtrain.iat[sample,2])
  y_pred_train.append(y_p_tr)

#Model Prediction on Testing Data

In [12]:
y_pred_test = []
for sample in range(Xtest.shape[0]):
  y_p_te = (m[0] * Xtest.iat[sample,0]) + c + (m[1] * Xtest.iat[sample,1])+(m[2]*Xtest.iat[sample,2])
  y_pred_test.append(y_p_te)

#DeNormalization

In [13]:
#Denormalize
Xtrain['L-3']=Xtrain['L-3']*(x1max-x1min)+x1min
Xtrain ['L-2']=Xtrain ['L-2']*(x2max-x2min)+x2min
Xtrain ['L-1']=Xtrain ['L-1']*(x3max-x2min)+x3min
Ytrain=Ytrain*(ymax-ymin)+ymin
Xtest['L-3']=Xtest['L-3']*(x1max-x1min)+x1min
Xtest['L-3']=Xtest['L-2']*(x2max-x2min)+x2min
Xtest['L-3']=Xtest['L-1']*(x3max-x3min)+x3min
Ytest=Ytest*(ymax-ymin)+ymin
y_pred_train=np.multiply(y_pred_train,(ymax-ymin)+ymin)
y_pred_test=np.multiply(y_pred_test,(ymax-ymin)+ymin)

#Error Metrics Calculation

In [14]:
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error


#Training Accuracies
rmse = math.sqrt(mean_squared_error(y_pred_train, Ytrain)) 
print('Root mean square error', rmse) 
mse = (mean_squared_error(y_pred_train,Ytrain)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_pred_train,Ytrain)
print('Mean absolute error', mae)


#Testing Accuracies
rmse = math.sqrt(mean_squared_error(y_pred_test,Ytest )) 
print('Root mean square error', rmse) 
mse = (mean_squared_error(y_pred_test,Ytest))
print('Mean square error', mse) 
mae=mean_absolute_error(y_pred_test,Ytest)
print('Mean absolute error', mae)

Root mean square error 2090.4082757029023
Mean square error 4369806.759127181
Mean absolute error 1938.1202257508608
Root mean square error 2006.2398474392332
Mean square error 4024998.3254529983
Mean absolute error 1853.1714421302995


#Deployment

In [16]:
X1=input('enter X1 value')
X2=input('enter X2 value')
X3=input ('enter X3 value')
print('X1=',X1)
print('X2=',X2)
print ('x3=',X3)
Y=m1*float(X1)+m2*float(X2)+m3*float(X3)+c
print('Y=',Y)

enter X1 value5024
enter X2 value4972
enter X3 value5208
X1= 5024
X2= 4972
x3= 5208
Y= 3528.5861938658873
